In [ ]:
function left_euler_left_euler(t0::Number, x0::Number, y0::Number, tf::Number, N::Integer)
    # t0 = initial time
    # tf = final time
    # x0 = initial value of x
    # y0 = initial value of y
    # N = number of time steps
    xa = zeros(typeof(t0),N+1) #declare N+1 member arrays for the solution
    ya = zeros(typeof(t0),N+1)
    ta = zeros(typeof(t0),N+1)
    h = (tf - t0)/N
    xa[1] = x0
    ya[1] = y0
    ta[1] = t0
    for k = 1:N
      xa[k+1] = xa[k] + h * ya[k]
      ya[k+1] = ya[k] - h * xa[k]
      ta[k+1] = t0 + h*k  
    end
    ta, xa, ya
end

Let's try solving on the interval $[0, 100]$ using $10^5$ steps. The initial conditions are
$x(0) = 1$ and $y(0)=0.$  The true solution is 
$$
   x(t) = \cos(t), \quad y(t) = \sin(t).
$$

In [ ]:
N = 10^5;

In [ ]:
t0 = 0.0;

In [ ]:
tf = 100.0;

In [ ]:
x0 = 1.0;

In [ ]:
y0 = 0.0;

In [ ]:
Ta, Xa, Ya = left_euler_left_euler(t0, x0, y0, tf, N);

In [ ]:
using Gadfly

In [ ]:
plot(layer(x=Ta, y=Xa, Geom.line, color=[colorant"purple"]), 
     layer(x -> cos(x),t0, tf, Geom.line, color=[colorant"black"]))

The graph matches the true solution. Should we claim success? Let's try solving on interval $[0,10^3]$ and increase $N$ to $10^6$. Thus we keep the same step size.

In [ ]:
N = 10^6;

In [ ]:
tf = 1000.0;

In [ ]:
Ta, Xa, Ya = left_euler_left_euler(t0, x0, y0, tf, N);

The graph is a bit of a mess, but it looks unpromising:

In [ ]:
plot(layer(x=Ta, y=Xa, Geom.line, color=[colorant"purple"]), 
     layer(x -> cos(x),t0, tf, Geom.line, color=[colorant"black"]))

Looking at the solution from $8 \times 10^5$ to $10^6$ steps shows that the amplitude has grown to over $1.5$. This is bad!

In [ ]:
plot(layer(x=Ta[8*10^5:10^6], y=Xa[8*10^5:10^6], Geom.line, color=[colorant"purple"]))     

Using the right-point rule to integrate gives the method
$$
  x_{k+1} = x_k + h  y_{k+1}, \quad   y_{k+1} = y_k - h  x_{k+1}.
$$
Although this method is  implicit, we can solve for $x_{k+1}$ and $y_{k+1}$ This gives
$$
   x_{k+1} = \frac{x_k + h y_k}{1+h^2}, \quad y_{k+1} = \frac{y_k - h x_k}{1+h^2}.
$$

In [ ]:
function right_euler_right_euler(t0::Number, x0::Number, y0::Number, t1::Number, N::Integer)
    xa = zeros(typeof(t0),N+1)
    ya = zeros(typeof(t0),N+1)
    ta = zeros(typeof(t0),N+1)
    h = (t1 - t0)/N
    xa[1] = x0
    ya[1] = y0
    ta[1] = t0
    for k = 1:N
      xa[k+1] = (xa[k] + h*ya[k])/(1+h^2)
      ya[k+1] = (ya[k] - h*xa[k])/(1+h^2)
      ta[k+1] = t0 + h*k  
    end
    ta, xa, ya
end

In [ ]:
Ta, Xa, Ya = right_euler_right_euler(t0, x0, y0, tf, N);

He-he! Before, the amplitude grew, now it shrinks!

In [ ]:
plot(layer(x=Ta[8*10^5:10^6], y=Xa[8*10^5:10^6], Geom.line, color=[colorant"purple"])) 

In [ ]:
function left_euler_right_euler(t0::Number, x0::Number, y0::Number, tf::Number, N::Integer)
    xa = zeros(typeof(t0),N+1)
    ya = zeros(typeof(t0),N+1)
    ta = zeros(typeof(t0),N+1)
    h = (tf - t0)/N
    xa[1] = x0
    ya[1] = y0
    ta[1] = t0
    for k = 1:N
      xa[k+1] = xa[k] + h*ya[k]
      ya[k+1] = ya[k] - h*xa[k+1]
      ta[k+1] = t0 + h*k 
    end
    ta, xa, ya
end

In [ ]:
Ta, Xa, Ya = left_euler_right_euler(t0, x0, y0, tf, N);

Success! The amplitude remains at one! 

In [ ]:
plot(layer(x=Ta[8*10^5:10^6], y=Xa[8*10^5:10^6], Geom.line, color=[colorant"purple"]))     

In [ ]:
function left_euler_right_euler(Q::Function, t0::Number, x0::Number, y0::Number, tf::Number, N::Integer)
    xa = zeros(typeof(t0),N+1)
    ya = zeros(typeof(t0),N+1)
    ta = zeros(typeof(t0),N+1)
    h = (tf - t0)/N
    xa[1] = x0
    ya[1] = y0
    ta[1] = t0
    for k = 1:N
      xa[k+1] = xa[k] + h*ya[k]
      ya[k+1] = ya[k] + h*Q(xa[k+1])
      ta[k+1] = t0 + h*k 
    end
    ta, xa, ya
end

Let's try this on the DE
$$
   \frac{\mathrm{d}^2 x}{\mathrm{d} t^2} =  -x^3
$$
We'll use the initial conditions
$$
  x(0) = 0, \quad x'(0) = 1
$$
A little known fact is that the solution of this IVP has a solution in terms of the Jacobi elliptic function $sn$. It is
$$
   x(t) = 2^{1/4} \mathrm{sn}(t / 2^{1/4}, -1).
$$

In [ ]:
t0  = 0.0

In [ ]:
x0 = 0.0

In [ ]:
y0 = 1.0

In [ ]:
tf = 25.0

In [ ]:
Q = x -> -x^3

In [ ]:
N = 10^6

In [ ]:
using Pkg

In [ ]:
Pkg.add("Elliptic")

In [ ]:
using Elliptic

In [ ]:
Ta, Xa, Ya = left_euler_right_euler(Q, t0, x0, y0, tf, N);

In [ ]:
plot(layer(x=Ta, y=Xa, Geom.line, color=[colorant"purple"]), 
     layer(x -> 2.0^(1/4) * Jacobi.sn(x/2.0^(1/4),-1),t0, tf, Geom.line, color=[colorant"black"]))

In [ ]:
Q = x -> 6*x^3 - x^5

In [ ]:
Ta, Xa, Ya = left_euler_right_euler(Q, t0, x0, y0, tf, N);

In [ ]:
plot(x=Ta, y=Xa, Geom.line, color=[colorant"purple"])
     